In [1]:
import sys
sys.path.append("../..")
from aips import get_engine
from IPython.display import display, HTML
from pyspark.sql import SparkSession
import ipywidgets as widgets
from PIL import Image
import pickle
import requests
import numpy
import torch
import clip
from io import BytesIO

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

engine = get_engine()
spark = SparkSession.builder.appName("AIPS").getOrCreate()

In [2]:
![ ! -d 'tmdb' ] && git clone --depth 1 https://github.com/ai-powered-search/tmdb.git
! cd tmdb && git pull
! cd tmdb && mkdir -p '../../../data/tmdb/' && tar -xvf movies_with_image_embeddings.tgz -C '../../../data/tmdb/'

Already up to date.
movies_with_image_embeddings.pickle


In [3]:
%run ../ch10/1.setup-the-movie-db.ipynb

Wiping "tmdb" collection
Creating "tmdb" collection
Status: Success
Adding LTR QParser for tmdb collection
Adding LTR Doc Transformer for tmdb collection
../../data/judgments.tgz already exists
../../data/movies.tgz already exists
Successfully written 65616 documents


## Listing 15.14

In [4]:
from aips.spark import create_view_from_collection
from aips.spark.dataframe import from_sql
from pyspark.sql.functions import split, regexp_replace, col

def normalize_embedding(embedding):
    return numpy.divide(embedding,
      numpy.linalg.norm(embedding,axis=0)).tolist()

def read(cache_name):
    cache_file_name = f"../../data/tmdb/{cache_name}.pickle"
    with open(cache_file_name, "rb") as fd:
        return pickle.load(fd)

def tmdb_with_embeddings_dataframe():
    movies = read("movies_with_image_embeddings")
    embeddings = movies["image_embeddings"]
    normalized_embeddings = [normalize_embedding(e) for e in embeddings]
    movies_dataframe = spark.createDataFrame(
        zip(movies["movie_ids"], movies["titles"], 
            movies["image_ids"], normalized_embeddings),
        schema=["movie_id", "title", "image_id", "image_embedding"])
    return movies_dataframe

def tmdb_lexical_embeddings_dataframe():
# NOT CURRENTLY WORKING. COMMENTING OUT THIS PARTIAL REFACTOR FOR NOW.
#    lexical_tmdb_collection = engine.get_collection("tmdb")
#    create_view_from_collection(lexical_tmdb_collection, "tmdb")
#    movies_dataframe = from_sql("SELECT id, overview, poster_file, poster_path FROM tmdb")
#    embeddings_dataframe = tmdb_with_embeddings_dataframe()
#    columns = ["id", "title", "movie_id", "image_embedding", 
#               "overview", "poster_file", "poster_path"]
#    joined = movies_dataframe.join(embeddings_dataframe, on=movies_dataframe.id == embeddings_dataframe.movie_id,
#                          how="left").select(*columns)
#    return joined
    lexical_tmdb_collection = engine.get_collection("tmdb")
    create_view_from_collection(lexical_tmdb_collection, "tmdb_lexical")
    embeddings_tmdb_collection = engine.get_collection("tmdb_with_embeddings")
    create_view_from_collection(embeddings_tmdb_collection, "tmdb_with_embeddings")
    
    #embedding_udf = spark.udf.register("embedding", lambda x: encode_text(x))
    #SELECT lexical.*, embeddings.image_embedding, embeddings.image_id, embeddings.movie_id, embedding(concat(lexical.title, ": ", lexical.overview)) as content_embedding
    joined_collection_sql = f"""    
    SELECT lexical.id id, embeddings.movie_id movie_id, lexical.title title, lexical.overview overview, embeddings.image_embedding, embeddings.image_id
    FROM tmdb_with_embeddings embeddings
    INNER JOIN (SELECT DISTINCT image_id from (SELECT movie_id, MIN(image_id) image_id from tmdb_with_embeddings GROUP BY movie_id ORDER BY image_id ASC)) distinct_images on embeddings.image_id = distinct_images.image_id
    INNER JOIN tmdb_lexical lexical ON lexical.id = embeddings.movie_id
    ORDER by lexical.id asc
    """
    #collection = engine.create_collection("tmdb_lexical_plus_embeddings")
    joined_dataframe = from_sql(joined_collection_sql) 
    joined_dataframe = joined_dataframe.withColumn("image_embedding", split(regexp_replace(col("image_embedding"),"\"",""), ","))
    return joined_dataframe
    
def create_embedding_indexes():
    embeddings_dataframe = tmdb_with_embeddings_dataframe()
    embeddings_collection = engine.create_collection("tmdb_with_embeddings")
    embeddings_collection.write(embeddings_dataframe)
    
    lexical_embeddings = tmdb_lexical_embeddings_dataframe()
    lexical_collection = engine.create_collection("tmdb_lexical_plus_embeddings")
    lexical_collection.write(lexical_embeddings)

In [5]:
create_embedding_indexes()

Wiping "tmdb_with_embeddings" collection
Creating "tmdb_with_embeddings" collection
Status: Success
Successfully written 7549 documents
Wiping "tmdb_lexical_plus_embeddings" collection
Creating "tmdb_lexical_plus_embeddings" collection
Status: Success
Successfully written 908 documents


## Listing 15.15

In [6]:
def load_image(full_path, log=False):   
    try:
        if full_path.startswith("http"):
            response = requests.get(full_path)
            image = Image.open(BytesIO(response.content))
        else:
            image = Image.open(full_path)
        if log: print("File Found")
        return image
    except:
        if log: print(f"No Image Available {full_path}")
        return []

def movie_search(query_embedding, limit=8):
    collection = engine.get_collection("tmdb_with_embeddings")
    request = {
        "query": query_embedding,
        "query_fields": ["image_embedding"],
        "return_fields": ["movie_id", "title", "image_id", "score"],
        "limit": limit,
        "quantization_size": "FLOAT32"}
    return collection.search(**request)
    
def encode_text(text, normalize=True):
    text = clip.tokenize([text]).to(device)    
    text_features = model.encode_text(text)
    embedding = text_features.tolist()[0] 
    if normalize:
        embedding = normalize_embedding(embedding)
    return embedding
    
def encode_image(image_file, normalize=True):
    image = load_image(image_file)
    inputs = preprocess(image).unsqueeze(0).to(device)
    embedding = model.encode_image(inputs).tolist()[0]
    if normalize:
        embedding = normalize_embedding(embedding)
    return embedding

def encode_text_and_image(text_query, image_file):    
    text_embedding = encode_text(text_query, True)
    image_embedding = encode_image(image_file, True)  
    return numpy.average((normalize_embedding(
        [text_embedding, image_embedding])), axis=0).tolist()

## Listing 15.16

In [7]:
def get_html(search_results):
    css = """
      <style type="text/css">
        .results { 
          margin-top: 15px; 
          display: flex; 
          flex-wrap: wrap; 
          justify-content: space-evenly; }
        .results .result { height: 250px; margin-bottom: 5px; }
      </style>"""
    
    results_html = ""
    for movie in search_results["docs"]:
        image_file = f"http://image.tmdb.org/t/p/w780/{movie['image_id']}.jpg"
        movie_link = f"https://www.themoviedb.org/movie/{movie['movie_id']}"
        img_html = f"<img title='{movie['title']}' class='result' src='{image_file}'>"
        results_html += f"<div>{movie['title']}<br/>(score: {movie['score']})<br/>"
        results_html += f"<a href='{movie_link}' target='_blank'>{img_html}</a></div>"
    return f"{css}<div class='results'>{results_html}</div>"
   
def display_results(search_results):    
    output = widgets.Output()
    with output:
        display(HTML(get_html(search_results))) 
    display(widgets.HBox(layout=widgets.Layout(justify_content="center")), output)   

def search_and_display(text_query="", image_query=None):
    if image_query:
        if text_query:
            query_embedding = encode_text_and_image(text_query, image_query)
        else:
            query_embedding = encode_image(image_query)
    else:
        query_embedding = encode_text(text_query)
    display_results(movie_search(query_embedding))

# Figure 15.5

In [8]:
search_and_display(text_query="singing in the rain")

HBox(layout=Layout(justify_content='center'))

Output()

In [9]:
search_and_display(text_query="superhero flying")

HBox(layout=Layout(justify_content='center'))

Output()

# Figure 15.6

In [10]:
search_and_display(text_query="superheroes flying")

HBox(layout=Layout(justify_content='center'))

Output()

# Figure 15.7

In [11]:
search_and_display(image_query="delorean-query.jpg")

HBox(layout=Layout(justify_content='center'))

Output()

# Figure 15.8

In [12]:
search_and_display(text_query="superhero", image_query="delorean-query.jpg")

HBox(layout=Layout(justify_content='center'))

Output()

# Listing 15.17

In [21]:
query = "singing in the rain"
query_embedding = encode_text(query)


def display_lexical_vs_vector_search_results(query_text):
    query_embedding = encode_text(query_text)
    collection = engine.get_collection("tmdb_lexical_plus_embeddings")
    over_request_limit = 10

    base_query = {
            "return_fields": ["id", "title", "id", "image_id", "movie_id", "score", "image_embedding"],
            "limit": over_request_limit
    }

    lexical_search = {"query":query_text, 
                      "query_fields": ["title","overview"],       
                      **base_query}


    vector_search = {"query":query_embedding, 
                     "query_fields": ["image_embedding"],
                     "quantization_size": "FLOAT32",
                     **base_query}

    lexical_search_results = collection.search(**lexical_search) 
    vector_search_results = collection.search(**vector_search)

    print(f"Lexical Query: {query_text}")
    #print(collection.transform_lexical_request(**lexical_search))
    display_results(lexical_search_results)

    print(f"Vector Query: {query_embedding[0:3]} ... {query_embedding[-3:]}")
    display_results(vector_search_results)
    
display_lexical_vs_vector_search_results(query)

Lexical Query: singing in the rain


HBox(layout=Layout(justify_content='center'))

Output()

Vector Query: [0.04053346811188779, 0.026027961337741636, -0.03153954410778869] ... [-0.0342272163116795, 0.050831037711545714, -0.08007933128422229]


HBox(layout=Layout(justify_content='center'))

Output()

In [22]:
self = collection

def hybrid_search(lexical_search_args, vector_search_args, limit=None, algorithm="rrf", algorithm_params={}):
    hybrid_search_results = None
    match algorithm:
        case "rrf":
            k = 60
            if "k" in algorithm_params: k = algorithm_params["k"]
            lexical_search_results = self.search(**lexical_search_args)
            vector_search_results = self.search(**vector_search_args)
            hybrid_search_scores = reciprocal_rank_fusion(k, 
                                       lexical_search_results["docs"], 
                                       vector_search_results["docs"])
            
            lexical_fields = {item["id"]: item for item in lexical_search_results["docs"]}
            vector_fields = {item["id"]: item for item in vector_search_results["docs"]}
            
            merged_search_docs = sorted([
                dict(lexical_fields[id], score=hybrid_search_scores[id]) \
                if id in lexical_fields \
                else dict(vector_fields[id], score=hybrid_search_scores[id]) \
                for id in hybrid_search_scores], key=lambda x: x["score"], reverse=True)
            
            if limit and limit < len(merged_search_docs): 
                merged_search_docs = merged_search_docs[:limit] 
                
            hybrid_search_results = {"docs": merged_search_docs }
        case "rerank":
            pass #need rerank implemented on coll
    return hybrid_search_results

def reciprocal_rank_fusion(k, *search_results):
    scores = {}
    for ranked_docs in search_results:
        for rank, doc in enumerate(ranked_docs, 1):
            scores[doc["id"]] = scores.get(doc["id"], 0)  + (1.0 / (k + rank))    
    return scores
    
    
# where
# k is a ranking constant
# q is a query in the set of queries
# d is a document in the result set of q
# result(q) is the result set of q
# rank( result(q), d ) is d's rank within the result(q) starting from 1

In [29]:
def display_hybrid_search_results(text_query):
    hybrid_search_results = hybrid_search(lexical_search, vector_search, limit=10)
    
    print(f"Lexical Query: {lexical_search['query']}")
    print(f"Vector Query: {vector_search['query'][0:3]} ... {vector_search['query'][-3:]}")
    display_results(hybrid_search_results)
    
display_hybrid_search_results(query)

Lexical Query: singing in the rain
Vector Query: [0.04053346811188779, 0.026027961337741636, -0.03153954410778869] ... [-0.0342272163116795, 0.050831037711545714, -0.08007933128422229]


HBox(layout=Layout(justify_content='center'))

Output()

In [26]:
query="lord of the rings"
display_lexical_vs_vector_search_results(query)
display_hybrid_search_results(query)

Lexical Query: lord of the rings


HBox(layout=Layout(justify_content='center'))

Output()

Vector Query: [-0.02987002229584486, -0.011187172293046544, 0.045517265720235234] ... [0.030793413730259785, -0.0029269506656306003, 0.00576058313400904]


HBox(layout=Layout(justify_content='center'))

Output()

Lexical Query: singing in the rain
Vector Query: [0.04053346811188779, 0.026027961337741636, -0.03153954410778869] ... [-0.0342272163116795, 0.050831037711545714, -0.08007933128422229]


HBox(layout=Layout(justify_content='center'))

Output()